# NEWCASTLE v. LEICESTER - match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [41]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""

!!! INPUT TEAMS HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

"""
home_side = "Newcastle United"
away_side = "Leicester"
matchday = "2024-12-14"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Newcastle United v. Leicester


### Calculate Projected Goals

In [42]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.1428571428571428 1.1428571428571428
1.4285714285714286 2.571428571428572
1.63 1.38
0.7011393514461 1.5775635407537254 1.0351966873706004 0.8281573498964804

 ================== projected_goals ==================== 

HOME Newcastle United projected goals: 1.8029297608614003
AWAY Leicester projected goals:  1.1830819284235434


### Poisson Distribution of Goal Probabilities

In [43]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Newcastle United v. Leicester [[5.04883991e-02 9.10270374e-02 8.20576774e-02 4.93147429e-02
  2.22277544e-02 8.01501598e-03 2.40841847e-03 6.20315621e-04
  1.39798187e-04]
 [5.97319126e-02 1.07692443e-01 9.70809552e-02 5.83433811e-02
  2.62972545e-02 9.48242057e-03 2.84935637e-03 7.33884201e-04
  1.65392708e-04]
 [3.53338732e-02 6.37044915e-02 5.74273618e-02 3.45124999e-02
  1.55559033e-02 5.60924020e-03 1.68551102e-03 4.34122568e-04
  9.78365622e-05]
 [1.39342889e-02 2.51225442e-02 2.26470913e-02 1.36103716e-02
  6.13463603e-03 2.21206357e-03 6.64699208e-04 1.71200855e-04
  3.85828895e-05]
 [4.12135136e-03 7.43050702e-03 6.69834112e-03 4.02554618e-03
  1.81444425e-03 6.54263109e-04 1.96598405e-04 5.06361594e-05
  1.14116798e-05]
 [9.75179262e-04 1.75817971e-03 1.58493727e-03 9.52510189e-04
  4.29327242e-04 1.54809372e-04 4.65184041e-05 1.19813450e-05
  2.70019044e-06]
 [1.92286160e-04 3.46678441e-04 3.12518439e-04 1.87816265e-04
  8.46548835e-05 3.05253618e-05 9.17251387e-06 2.3624854

### Under Market

In [44]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Newcastle United v. Leicester
    Goals      Prob  Implied Odds
0    0.5  0.050488     19.806530
1    1.5  0.201247      4.969010
2    2.5  0.426331      2.345593
3    3.5  0.650366      1.537596


In [45]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Newcastle United v. Leicester
    Goals      Prob  Implied Odds
0    0.5  0.949396      1.053301
1    1.5  0.798637      1.252133
2    2.5  0.573553      1.743517
3    3.5  0.349519      2.861077


### 1 x 2 Probability

In [46]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                  winner      prob  implied_odds
0  Newcastle United (1)  0.518857      1.927314
1              Draw (x)  0.231197      4.325308
2         Leicester (2)  0.249830      4.002716


# Full Results Summary:

In [51]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):
    translations = {
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City"
    }
    if string in translations:
        return translations["string"]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
filtered_odds = odds_df.loc[odds_df["match"] == match_string]
implied = {
    "1": 1 / filtered_odds["1"],
    "x": 1 / filtered_odds["x"],
    "2": 1 / filtered_odds["2"],
    "point": filtered_odds["ovr_und_point"],
    "over": 1 / filtered_odds["over"],
    "under": 1 / filtered_odds["under"]
}
implied_odds_df = pd.DataFrame(implied)

print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)

CALCULATED PROBABILITIES FOR Newcastle United v. Leicester City


        ~ ~ moneyline market ~ ~

                  winner      prob  implied_odds
0  Newcastle United (1)  0.518857      1.927314
1              Draw (x)  0.231197      4.325308
2         Leicester (2)  0.249830      4.002716


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.949396      1.053301
1    1.5  0.798637      1.252133
2    2.5  0.573553      1.743517
3    3.5  0.349519      2.861077


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.050488     19.806530
1    1.5  0.201247      4.969010
2    2.5  0.426331      2.345593
3    3.5  0.650366      1.537596

 ~~ MyBookie Odds for Newcastle United v. Leicester City ~~
       1    x    2  ovr_und_point  over  under
3  1.29  5.7  8.3            3.5  2.06   1.71

 ~~ MyBookie Odds IMPLIED for Newcastle United v. Leicester City ~~
           1         x         2  point      over     under
3  0.775194  0.175439  

# Conclusion and Pick

SAFE PICK: Leicester is playing poorly and Newcastle as the home side are highly favored by MyBookie. We Newcastle at a 51% chance of winning. I don't like the odds at 1.29 however.

RISKIER PICK: I have not earned much from making Under bets historically. That's why I consider this risky. We have the under 3.5 at 65% chance of hitting, slightly higher than the MB implied at 58%. I like the odds at 1.71

### My Pick:

**U 3.5 @ 1.71**